In [29]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn import preprocessing
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        read_data = pd.read_csv(os.path.join(dirname, filename))

#exploring data
print(read_data.head())

#independent training values
x_values = read_data["review"]
print(x_values[1:10])

#dependent values
label_encoder = preprocessing.LabelEncoder()
y_values = label_encoder.fit_transform(read_data['sentiment'])
print(y_values[0:10])

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
1    A wonderful little production. <br /><br />The...
2    I thought this was a wonderful way to spend ti...
3    Basically there's a family where a little boy ...
4    Petter Mattei's "Love in the Time of Money" is...
5    Probably my all-time favorite movie, a story o...
6    I sure would like to see a resurrection of a u...
7    This show was an amazing, fresh & innovative i...
8    Encouraged by the positive comments about this...
9    If you like original gut wrenching laughter yo...
Name: review, dtype: object
[1 1 1 0 1 1 1 0 0 1]


In [30]:
stopwords = set(stopwords.words('english'))
print(stopwords)

splitted_list = x_values[0].split(" ")
print(splitted_list)

{"needn't", "you're", 'because', 'during', 'whom', 'wasn', "she's", 'that', "hasn't", 'isn', 'she', 'having', 'out', 'their', 'wouldn', 'just', 'our', 'd', 'before', 'itself', "isn't", 'his', 'there', 'each', "wasn't", 'than', "don't", 'it', 'under', 'doing', 'doesn', 'didn', 'being', 's', "mustn't", 'couldn', 'so', 't', 'only', 'other', "mightn't", "didn't", 'about', 'myself', 'or', 'of', "hadn't", "that'll", 'he', 'between', 'to', 'those', 'has', 'any', "should've", 'can', 'what', 'here', 'himself', 'been', 've', "wouldn't", 'off', 'themselves', 'me', 'more', 'own', 'as', 'but', 'its', 'too', 'had', 'most', 'my', 'all', 'mustn', 'when', 'not', 'him', 'in', 'i', "weren't", 'nor', 'an', 'on', 'aren', "couldn't", 'and', 'yourselves', 'a', 'shouldn', 'haven', "shouldn't", 'against', 'then', 'your', 'will', "haven't", 'after', "aren't", "it's", 'hasn', "you've", "shan't", 'mightn', 'll', 'was', 'theirs', 'is', 'such', 'below', 'both', 'the', 'hadn', 'for', 'does', 'hers', 'they', 'through

In [31]:
from bs4 import BeautifulSoup
for i,word_data in enumerate(x_values):
    x_values[i] = BeautifulSoup(x_values[i], 'lxml').get_text()
    splitted_values = x_values[i].split(" ")
    x_values[i] = []
    for soft_words_split in splitted_values:
        if soft_words_split not in stopwords:
            x_values[i].append(soft_words_split + " ")
            
print(x_values[0])

['One ', 'reviewers ', 'mentioned ', 'watching ', '1 ', 'Oz ', 'episode ', 'hooked. ', 'They ', 'right, ', 'exactly ', 'happened ', 'me.The ', 'first ', 'thing ', 'struck ', 'Oz ', 'brutality ', 'unflinching ', 'scenes ', 'violence, ', 'set ', 'right ', 'word ', 'GO. ', 'Trust ', 'me, ', 'show ', 'faint ', 'hearted ', 'timid. ', 'This ', 'show ', 'pulls ', 'punches ', 'regards ', 'drugs, ', 'sex ', 'violence. ', 'Its ', 'hardcore, ', 'classic ', 'use ', 'word.It ', 'called ', 'OZ ', 'nickname ', 'given ', 'Oswald ', 'Maximum ', 'Security ', 'State ', 'Penitentary. ', 'It ', 'focuses ', 'mainly ', 'Emerald ', 'City, ', 'experimental ', 'section ', 'prison ', 'cells ', 'glass ', 'fronts ', 'face ', 'inwards, ', 'privacy ', 'high ', 'agenda. ', 'Em ', 'City ', 'home ', 'many..Aryans, ', 'Muslims, ', 'gangstas, ', 'Latinos, ', 'Christians, ', 'Italians, ', 'Irish ', 'more....so ', 'scuffles, ', 'death ', 'stares, ', 'dodgy ', 'dealings ', 'shady ', 'agreements ', 'never ', 'far ', 'away.I 

In [32]:
#tokenize
#give input to embedding layer

from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence

encoded_words = x_values;
vocab_size = len(x_values[0])
print(vocab_size)
# integer encode the document
listToStr = ' '.join([str(elem) for elem in x_values[0]])
result = one_hot(listToStr, round(vocab_size*1.3))
print(result)

for i,values in enumerate(x_values):
    listToStr = ' '.join([str(elem) for elem in x_values[i]])
    vocab_size = len(x_values[i])
    encoded_words[i] = one_hot(listToStr, round(vocab_size*1.3))



179
[37, 36, 168, 194, 202, 78, 221, 77, 71, 88, 71, 207, 113, 109, 217, 69, 154, 78, 58, 110, 32, 190, 210, 88, 185, 122, 118, 113, 196, 69, 128, 27, 200, 196, 76, 101, 39, 11, 106, 190, 184, 162, 214, 212, 185, 194, 211, 78, 163, 21, 226, 208, 87, 38, 15, 194, 65, 9, 232, 202, 149, 92, 219, 45, 221, 76, 90, 128, 60, 225, 196, 207, 202, 3, 7, 62, 93, 123, 61, 111, 156, 88, 49, 13, 85, 117, 73, 37, 199, 14, 207, 36, 24, 5, 74, 129, 152, 194, 232, 196, 109, 79, 199, 161, 117, 136, 19, 214, 183, 119, 193, 136, 69, 136, 95, 78, 167, 191, 109, 217, 221, 74, 209, 117, 154, 108, 98, 74, 152, 74, 5, 194, 74, 14, 49, 74, 126, 164, 78, 196, 116, 225, 166, 146, 190, 101, 190, 120, 56, 57, 109, 62, 188, 8, 109, 20, 199, 47, 5, 194, 37, 172, 202, 62, 8, 54, 219, 95, 109, 75, 1, 150, 219, 180, 194, 78, 152, 74, 20, 171, 182, 182, 47, 3, 133, 177]


In [33]:
print(len(encoded_words[0]))

max_strings_length = 0
for values in encoded_words:
    if max_strings_length < len(values): max_strings_length = len(values)

X_data = sequence.pad_sequences(encoded_words, maxlen=max_strings_length)

x_train, x_test, y_train, y_test = train_test_split(X_data, y_values, test_size=0.33)

from keras.layers import LSTM
from keras.layers import Dropout

model = Sequential()
model.add(Embedding(5000,32,input_length = max_strings_length))
model.add(LSTM(200, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=25)


186
Epoch 1/25
1047/1047 [==============================] - 259s 247ms/step - loss: 0.6935 - accuracy: 0.5038 - val_loss: 0.6933 - val_accuracy: 0.4935
Epoch 2/25
1047/1047 [==============================] - 258s 247ms/step - loss: 0.6922 - accuracy: 0.5194 - val_loss: 0.6942 - val_accuracy: 0.5113
Epoch 3/25
1047/1047 [==============================] - 261s 249ms/step - loss: 0.6896 - accuracy: 0.5331 - val_loss: 0.6953 - val_accuracy: 0.5062
Epoch 4/25
1047/1047 [==============================] - 262s 250ms/step - loss: 0.6866 - accuracy: 0.5430 - val_loss: 0.6931 - val_accuracy: 0.5093
Epoch 5/25
1047/1047 [==============================] - 266s 254ms/step - loss: 0.6821 - accuracy: 0.5507 - val_loss: 0.6954 - val_accuracy: 0.5106
Epoch 6/25
1047/1047 [==============================] - 265s 253ms/step - loss: 0.6774 - accuracy: 0.5590 - val_loss: 0.7016 - val_accuracy: 0.5129
Epoch 7/25
1047/1047 [==============================] - 263s 251ms/step - loss: 0.6729 - accuracy: 0.5624 - 

In [34]:
accuracy_score = model.evaluate(x_test,y_test, verbose=0)
print(accuracy_score)

[1.2775989770889282, 0.5119394063949585]


In [ ]:
import random

for i in range(9):
    RandomNumber = random.randint(0,len(x_test))
    sentiment = model.predict_classes(x_test[RandomNumber]) 
    print("sentence: %s and sentiment: %d",x_test[RandomNumber],sentiment)